## import, load

In [186]:
import pandas as pd

In [187]:
hr = pd.read_csv('HR.csv')

In [188]:
hr.left.value_counts()

0    11428
1     3571
Name: left, dtype: int64

In [189]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [190]:
# Label Encoder

In [191]:
hr
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE2 = LabelEncoder()

LE.fit(hr['Department'])
LE2.fit(hr['salary'])

list(LE.classes_)
list(LE2.classes_)

LE.transform(hr['Department'])
LE2.transform(hr['salary'])


array([1, 2, 2, ..., 1, 1, 1])

In [192]:
ap = pd.DataFrame()
ap['d'] = hr['Department']
ap['D'] = LE.transform(hr['Department'])
ap['S'] = LE2.transform(hr['salary'])
ap['s'] = hr['salary']

In [193]:
df = hr.copy()
df['Department'] = LE.transform(hr['Department'])
df['salary'] = LE2.transform(hr['salary'])
df

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,7,1
1,0.80,0.86,5,262,6,0,1,0,7,2
2,0.11,0.88,7,272,4,0,1,0,7,2
3,0.72,0.87,5,223,5,0,1,0,7,1
4,0.37,0.52,2,159,3,0,1,0,7,1
...,...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,1,0,8,1
14995,0.37,0.48,2,160,3,0,1,0,8,1
14996,0.37,0.53,2,143,3,0,1,0,8,1
14997,0.11,0.96,6,280,4,0,1,0,8,1


## feature selection

#### selection

In [194]:
from sklearn.feature_selection import SelectKBest, chi2


In [195]:
df
X = df.drop('left', axis = 'columns')
y = df['left']
fitted = SelectKBest(chi2, k=5).fit(X,y)

In [203]:
fs1 = pd.DataFrame()
fs1['score'] = fitted.scores_
fs1['features'] = X.columns
fs1.sort_values(by = 'score', ascending = False).head(7)

,score,features
3,945.588933,average_montly_hours
5,306.737710,Work_accident
0,228.193183,satisfaction_level
4,191.708547,time_spend_company
6,56.044865,promotion_last_5years
7,21.672355,Department
2,3.390205,number_project


#### importance

In [200]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(X,y)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [205]:
fs2 = pd.DataFrame()
fs2['features'] = X.columns
fs2['score'] = model.feature_importances_
fs2.sort_values(by='score', ascending = False).head(7)

,features,score
0,satisfaction_level,0.240250
2,number_project,0.236421
4,time_spend_company,0.183339
3,average_montly_hours,0.145101
1,last_evaluation,0.140065
7,Department,0.020854
8,salary,0.016662


In [222]:
features = df[['average_montly_hours', 'satisfaction_level', 'time_spend_company', 'left']]
features

,average_montly_hours,satisfaction_level,time_spend_company,left
0,157,0.38,3,1
1,262,0.80,6,1
2,272,0.11,4,1
3,223,0.72,5,1
4,159,0.37,3,1
...,...,...,...,...
14994,151,0.40,3,1
14995,160,0.37,3,1
14996,143,0.37,3,1
14997,280,0.11,4,1


## Prediction

In [224]:
X = features.drop('left', axis = 'columns')
y = features['left']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [225]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train, y_train)
log.score(X_test, y_test)

0.7523232323232323

In [227]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.7828282828282829

In [228]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
tree.score(X_test, y_test)

0.9678787878787879

In [230]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9791919191919192